Inference of a single count label c and n coupled labels $x_i,y_i$, $i=1,\ldots,n$, such that $c=x_i+y_i$, leads to 

$\max\limits_{c=x_1+x_2\atop {\cdots\atop c=x_n+y_n}} log(p(c))+\sum_{i=1}^n \big (log(P_x(x_i))+log(P_y(y_i))\big )$

In [1]:
import numpy as np

# create instance of the problem
n_wins = 6
n_max_events = 5
n_coupled_labels = 3

Pc = np.random.rand(n_max_events+1, n_wins)
Pc = Pc / np.sum( Pc, axis=0)
log_Pc = np.log( Pc )

log_P = []
for i in range(n_coupled_labels):
    Px = np.random.rand(n_max_events+1, n_wins)
    Px = Px / np.sum( Px, axis=0)

    Py = np.random.rand(n_max_events+1, n_wins)
    Py = Py / np.sum( Py, axis=0)
    
    log_P.append( (np.log(Px),np.log(Py)))


    
print(log_Pc)
print(log_P[0][0])
print(log_P[0][1])


[[-1.57970992 -1.30072108 -0.99899676 -0.87703923 -1.27565487 -1.41395495]
 [-2.04670449 -2.09639317 -4.0429285  -2.84569716 -1.52808199 -1.55412177]
 [-1.65038708 -2.0132812  -4.36893371 -3.47825736 -1.79449312 -2.03614054]
 [-1.54141548 -1.67852643 -3.0127867  -1.25389527 -3.1457398  -2.40710287]
 [-2.12528234 -2.52364258 -1.3920487  -1.94814216 -1.51613832 -2.04748625]
 [-1.97074792 -1.58767583 -1.19131822 -2.70153092 -2.59032441 -1.63076815]]
[[-2.56176678 -3.23541109 -1.83613237 -1.57846418 -1.56248615 -3.33076993]
 [-1.41678093 -1.15638639 -1.3577865  -1.85428374 -3.19427697 -1.95197663]
 [-3.5874685  -1.2529747  -1.84619191 -1.60527912 -2.08970264 -1.13841698]
 [-3.19898155 -3.15126988 -2.21250147 -1.75779771 -1.85938703 -1.65063222]
 [-2.0236712  -1.43232128 -2.43359047 -1.66003608 -1.37109455 -1.20905534]
 [-0.73460067 -2.54047026 -1.47686415 -2.60702699 -1.53215946 -4.46471502]]
[[-2.89093261 -2.01727277 -1.26762658 -1.38193568 -2.07660371 -4.43750343]
 [-1.31214096 -1.044033

In [2]:
def struct_inference( log_Pc, log_P ):
    
    n_events = log_Pc.shape[0]-1
    n_wins = log_Pc.shape[1]
    
    phi = []
    arg_phi = []
    score = np.copy( log_Pc )
    for i in range( len(log_P) ):
        log_Px = log_P[i][0]
        log_flip_Py = np.flipud( log_P[i][1] )

        phi_ = np.zeros( (n_events+1,n_wins) )
        arg_phi_ = np.zeros( (n_events+1,n_wins), dtype = int )
        for c in range( n_events+1):
            tmp = log_Px[0:c+1,:] + log_flip_Py[-(c+1):,:]
            arg_phi_[c,:] = np.argmax( tmp, axis=0 )
            #phi_[c,:] = np.max( tmp, axis=0)
            idx_row, idx_col = np.unravel_index(arg_phi_[c,:]*tmp.shape[1]+np.arange(0,tmp.shape[1] ), tmp.shape)
            phi_[c,:] = tmp[idx_row,idx_col]

        arg_phi.append(arg_phi_ )

        score += phi_
            
    c = np.argmax(score,axis=0)
        
    lab = []
    for i in range( len(log_P) ):
        idx_row, idx_col = np.unravel_index( c*n_wins+np.arange(0,n_wins ), (n_events+1,n_wins))
        lab.append( arg_phi[i][idx_row,idx_col])
     
    return c, lab


In [9]:
# inference  
c, lab = struct_inference( log_Pc, log_P )

print("number of windows:", log_Pc.shape[1])
print("max num of events in a ingle window:", log_Pc.shape[0]-1)
print("count_label:", c)
for i in range( len(log_P) ):
    print("coupled_labels",i,"   x:",lab[i], "y:", c-lab[i])

number of windows: 6
max num of events in a ingle window: 5
count_label: [2 5 5 3 4 5]
coupled_labels 0    x: [1 4 5 0 3 2] y: [1 1 0 3 1 3]
coupled_labels 1    x: [1 4 5 2 1 3] y: [1 1 0 1 3 2]
coupled_labels 2    x: [1 0 5 1 3 0] y: [1 5 0 2 1 5]
